<a href="https://colab.research.google.com/github/avolinski/Nengo/blob/main/nengo_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet nengo-dl